# Building Chatbots with AWS Bedrock: Hands-on Guide

Hello everyone! 👋

In this guide, I'll explain how to build a chatbot on AWS without any headaches.

## Project Context
This guide documents my exploration of AWS Bedrock for chatbot development within an existing AWS infrastructure. No vendor comparisons — just practical implementation steps. This guide based by official amazon-bedrock-workshop repository.

## 1. Setting Up: First API Call
Goal: Initialize a model and make a basic API call.

In [2]:
!pip install boto3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.4/14.4 MB 95.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.1 MB/s eta 0:00:00


In [5]:
import json
import botocore
import boto3
from IPython.display import display, Markdown
import time

Start with the region initialization.

Create a file with AWS credentials.

```
aws configure
```

Or create a file manually here

```
Linux/MacOS: ~/.aws/credentials
Windows: C:\Users\YOUR_USER\.aws\credentials
```

```
[default]
aws_access_key_id = YOUR_ACCESS_KEY_ID
aws_secret_access_key = YOUR_SECRET_ACCESS_KEY
region = us-east-1  # or yours region
```


### Modify your code to load settings

### 1. Using boto3 with a profile

```
import boto3
import json
from botocore.exceptions import ClientError

# Укажите профиль явно
session = boto3.Session(
    profile_name='default',
    region_name='us-east-1'  
```

### 2. Load from JSON

```
{
    "aws_access_key_id": "YOUR_ACCESS_KEY_ID",
    "aws_secret_access_key": "YOUR_SECRET_ACCESS_KEY",
    "region": "us-east-1"
}
```



In [17]:
json_file_path = '/content/aws_config.js'

In [19]:
import boto3
import json
import os

def load_aws_credentials(json_file_path):
    with open(json_file_path, 'r') as f:
        credentials = json.load(f)

    os.environ['AWS_ACCESS_KEY_ID'] = credentials['aws_access_key_id']
    os.environ['AWS_SECRET_ACCESS_KEY'] = credentials['aws_secret_access_key']
    os.environ['AWS_DEFAULT_REGION'] = credentials.get('region', 'us-east-1')

    return credentials

load_aws_credentials('aws_credentials.json')

bedrock = boto3.client('bedrock-runtime')

In [20]:
# Initialize Bedrock client
session = boto3.session.Session()
region = session.region_name
bedrock = boto3.client(service_name='bedrock-runtime', region_name="us-east-1")

In [21]:
MODELS = {
    "Claude 3.7 Sonnet": "us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    "Amazon Nova Pro": "us.amazon.nova-pro-v1:0",
    "Amazon Nova Micro": "us.amazon.nova-micro-v1:0",
}

In [22]:
# Utility function to display model responses in a more readable format
def display_response(response, model_name=None):
    if model_name:
        display(Markdown(f"### Response from {model_name}"))
    display(Markdown(response))
    print("\n" + "-"*80 + "\n")

## 2. Text Summarization with Foundation Models (basic Invoke API)

In [10]:
text_to_summarize = """
AWS took all of that feedback from customers, and today we are excited to announce Amazon Bedrock, \
a new service that makes FMs from AI21 Labs, Anthropic, Stability AI, and Amazon accessible via an API. \
Bedrock is the easiest way for customers to build and scale generative AI-based applications using FMs, \
democratizing access for all builders. Bedrock will offer the ability to access a range of powerful FMs \
for text and images—including Amazons Titan FMs, which consist of two new LLMs we're also announcing \
today—through a scalable, reliable, and secure AWS managed service. With Bedrock's serverless experience, \
customers can easily find the right model for what they're trying to get done, get started quickly, privately \
customize FMs with their own data, and easily integrate and deploy them into their applications using the AWS \
tools and capabilities they are familiar with, without having to manage any infrastructure (including integrations \
with Amazon SageMaker ML features like Experiments to test different models and Pipelines to manage their FMs at scale).
"""

 Each model family has its own distinct request and response format, you'll need to craft specific JSON payloads tailored to each model

In [11]:
prompt = f"""Please provide a summary of the following text. Do not add any information that is not mentioned in the text below.
<text>
{text_to_summarize}
</text>
"""

In [23]:
# Create request body for Claude 3.7 Sonnet
claude_body = json.dumps({
    "anthropic_version": "bedrock-2023-05-31",
    "max_tokens": 1000,
    "temperature": 0.5,
    "top_p": 0.9,
    "messages": [
        {
            "role": "user",
            "content": [{"type": "text", "text": prompt}]
        }
    ],
})

# Send request to Claude 3.7 Sonnet
try:
    response = bedrock.invoke_model(
        modelId=MODELS["Claude 3.7 Sonnet"],
        body=claude_body,
        accept="application/json",
        contentType="application/json"
    )
    response_body = json.loads(response.get('body').read())

    # Extract and display the response text
    claude_summary = response_body["content"][0]["text"]
    display_response(claude_summary, "Claude 3.7 Sonnet (Invoke Model API)")

except botocore.exceptions.ClientError as error:
    if error.response['Error']['Code'] == 'AccessDeniedException':
        print(f"\x1b[41m{error.response['Error']['Message']}\
            \nTo troubleshoot this issue please refer to the following resources.\
            \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
            \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
    else:
        raise error

### Response from Claude 3.7 Sonnet (Invoke Model API)

# Summary

Amazon announced Amazon Bedrock, a new service that provides API access to foundation models (FMs) from AI21 Labs, Anthropic, Stability AI, and Amazon. Bedrock aims to democratize access to generative AI by offering a simple way for customers to build and scale applications using these models. The service includes text and image models, including Amazon's new Titan LLMs. As a serverless AWS managed service, Bedrock allows customers to find appropriate models, get started quickly, customize models with their own data, and integrate them into applications using familiar AWS tools without managing infrastructure. The service integrates with Amazon SageMaker features like Experiments and Pipelines.


--------------------------------------------------------------------------------



## 2.2 Text Summarization using the Converse API

Invoke Model API has several limitations:
- it uses different request/response formats for each model family;
- there is no built-in support for multi-turn conversations;
- it requires custom handling for different model capabilities

Converse API addresses these limitations by providing a unified interface.

In [24]:
converse_request = {
    "messages": [
        {
            "role": "user",
            "content": [
                {
                    "text": f"Please provide a concise summary of the following text in 2-3 sentences. Text to summarize: {text_to_summarize}"
                }
            ]
        }
    ],
    "inferenceConfig": {
        "temperature": 0.4,
        "topP": 0.9,
        "maxTokens": 500
    }
}

In [25]:
try:
    response = bedrock.converse(
        modelId=MODELS["Claude 3.7 Sonnet"],
        messages=converse_request["messages"],
        inferenceConfig=converse_request["inferenceConfig"]
    )

    # Extract the model's response
    claude_converse_response = response["output"]["message"]["content"][0]["text"]
    display_response(claude_converse_response, "Claude 3.7 Sonnet (Converse API)")
except botocore.exceptions.ClientError as error:
    if error.response['Error']['Code'] == 'AccessDeniedException':
        print(f"\x1b[41m{error.response['Error']['Code']}: {error.response['Error']['Message']}\x1b[0m")
        print("Please ensure you have the necessary permissions for Amazon Bedrock.")
    else:
        raise error

### Response from Claude 3.7 Sonnet (Converse API)

AWS has launched Amazon Bedrock, a service providing API access to foundation models from AI21 Labs, Anthropic, Stability AI, and Amazon's own Titan models for text and image generation. Bedrock simplifies generative AI application development through a serverless experience, allowing customers to easily find, customize, and deploy models using familiar AWS tools without managing infrastructure.


--------------------------------------------------------------------------------



## 2.3 Converse API

Use the [Converse](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_runtime_Converse.html) or [ConverseStream](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_runtime_ConverseStream.html) API operations to send messages to a model. While you could use the standard [InvokeModel](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_runtime_InvokeModel.html) calls for chat applications, AWS recommends the Converse API instead. It offers a single, consistent interface that works across all Bedrock models supporting chat, letting you write your code once and reuse it with different models.

If a specific model has its own unique settings, the Converse API allows you to pass those as dedicated parameters. You can use this API to build conversational applications, like a chatbot that maintains a multi-turn dialogue. This is perfect for creating a custom assistant—for example, one with a specific persona, like a helpful tech support agent.

The Converse API also supports other core Bedrock features, including [tool](https://docs.aws.amazon.com/bedrock/latest/userguide/tool-use.html) use and content [guardrails](https://docs.aws.amazon.com/bedrock/latest/userguide/guardrails-use-converse-api.html).

The list of [all Amazon Bedrock models that support messages](https://docs.aws.amazon.com/bedrock/latest/userguide/converse-api.html).
Converse Documentation



```
{
  "modelId": "us.anthropic.claude-3-7-sonnet-20250219-v1:0", // Required: Model identifier
  
  "messages": [ // Required: Conversation history
    {
      "role": "user", // Who sent the message
      "content": [
        {
          "text": "Your prompt or message here" // Message content
        }
      ]
    }
  ],
  
  "system": [ // Optional: System instructions
    {
      "text": "You are a helpful AI assistant."
    }
  ],
  
  "inferenceConfig": { // Optional: Inference parameters
    "temperature": 0.7, // Randomness (0.0-1.0)
    "topP": 0.9, // Diversity control (0.0-1.0)
    "maxTokens": 2000, // Maximum response length
    "stopSequences": [] // Stop generation triggers
  },
  
  "toolConfig": { // Optional: Function calling setup
    "tools": [],
    "toolChoice": {
      "auto": {} // Let model decide when to use tools
    }
  }
}
```



## 2.4 Easily switch between models

One of the biggest advantages of the Converse API is the ability to easily switch between models using the exact same request format.

In [27]:
# call different models with the same converse request
results = {}
for model_name, model_id in MODELS.items(): # looping over all models defined above
        try:
            start_time = time.time()
            response = bedrock.converse(
                modelId=model_id,
                messages=converse_request["messages"],
                inferenceConfig=converse_request["inferenceConfig"] if "inferenceConfig" in converse_request else None
            )
            end_time = time.time()

            # Extract the model's response using the correct structure
            model_response = response["output"]["message"]["content"][0]["text"]
            response_time = round(end_time - start_time, 2)

            results[model_name] = {
                "response": model_response,
                "time": response_time
            }

            print(f"✅ Successfully called {model_name} (took {response_time} seconds)")

        except Exception as e:
            print(f"❌ Error calling {model_name}: {str(e)}")
            results[model_name] = {
                "response": f"Error: {str(e)}",
                "time": None
            }


✅ Successfully called Claude 3.7 Sonnet (took 2.64 seconds)
✅ Successfully called Amazon Nova Pro (took 2.11 seconds)
✅ Successfully called Amazon Nova Micro (took 0.77 seconds)


In [28]:
# Display results in a formatted way
for model_name, result in results.items():
    if "Error" not in result["response"]:
        display(Markdown(f"### {model_name} (took {result['time']} seconds)"))
        display(Markdown(result["response"]))
        print("-" * 80)

### Claude 3.7 Sonnet (took 2.64 seconds)

AWS has launched Amazon Bedrock, a new service providing API access to foundation models (FMs) from various AI companies, including Amazon's own Titan LLMs. The service democratizes generative AI by offering a serverless experience where customers can easily find, customize, and deploy text and image models without managing infrastructure. Bedrock integrates with existing AWS tools, allowing users to test different models and manage their FMs at scale.

--------------------------------------------------------------------------------


### Amazon Nova Pro (took 2.11 seconds)

AWS has launched Amazon Bedrock, a new service that provides easy access to Foundation Models (FMs) from various providers like AI21 Labs, Anthropic, Stability AI, and Amazon via an API, enabling customers to build and scale generative AI applications. Bedrock offers a serverless experience with scalable, reliable, and secure access to a range of powerful FMs for text and images, allowing users to customize, integrate, and deploy models using familiar AWS tools without managing infrastructure.

--------------------------------------------------------------------------------


### Amazon Nova Micro (took 0.77 seconds)

AWS has launched Amazon Bedrock, a new service that provides easy access to generative AI models from AI21 Labs, Anthropic, Stability AI, and Amazon via an API, enabling developers to quickly build and scale AI-based applications without managing infrastructure. Bedrock offers scalable, secure access to a range of powerful models for text and images, including Amazon's new Titan models, and integrates seamlessly with AWS tools like SageMaker.

--------------------------------------------------------------------------------


## 2.5 Cross-Regional Inference in Amazon Bedrock

Amazon Bedrock offers Cross-Regional Inference which automatically selects the optimal AWS Region within your geography to process your inference requests.

To use Cross-Regional Inference, you simply need to specify a cross-region inference profile as the modelId when making a request. Cross-region inference profiles are identified by including a region prefix (e.g., us. or eu.) before the model name.

```
{
    "Amazon Nova Pro": "amazon.nova-pro-v1:0",  # Regular model ID
    "Amazon Nova Pro (CRIS)": "us.amazon.nova-pro-v1:0"  # Cross-regional model ID
}
```

```
# Regular model invocation (standard region)
standard_response = bedrock.converse(
    modelId="anthropic.claude-3-5-sonnet-20240620-v1:0",  # Standard model ID
    messages=converse_request["messages"]
)

# Cross-region inference (note the "us." prefix)
cris_response = bedrock.converse(
    modelId="us.anthropic.claude-3-5-sonnet-20240620-v1:0",  # Cross-region model ID with regional prefix
    messages=converse_request["messages"]
)

# Print responses
print("Standard response:", standard_response["output"]["message"]["content"][0]["text"])
print("Cross-region response:", cris_response["output"]["message"]["content"][0]["text"])
```

## 2.6 Multi-turn Conversations

In [31]:
# Example of a multi-turn conversation with Converse API
multi_turn_messages = [
    {
        "role": "user",
        "content": [{"text": f"Please summarize this text: {text_to_summarize}"}]
    },
    {
        "role": "assistant",
        "content": [{"text": results["Claude 3.7 Sonnet"]["response"]}]
    },
    {
        "role": "user",
        "content": [{"text": "Can you make this summary even shorter, just 1 sentence?"}]
    }
]

try:
    response = bedrock.converse(
        modelId=MODELS["Claude 3.7 Sonnet"],
        messages=multi_turn_messages,
        inferenceConfig={"temperature": 0.2, "maxTokens": 500}
    )

    # Extract the model's response using the correct structure
    follow_up_response = response["output"]["message"]["content"][0]["text"]
    display_response(follow_up_response, "Claude 3.7 Sonnet (Multi-turn conversation)")

except Exception as e:
    print(f"Error: {str(e)}")

### Response from Claude 3.7 Sonnet (Multi-turn conversation)

Amazon Bedrock is AWS's new serverless service that provides API access to foundation models from multiple AI companies, allowing customers to easily find, customize, and deploy generative AI models without managing infrastructure.


--------------------------------------------------------------------------------



## 2.7 Streaming Responses with ConverseStream API

In [33]:
# Example of streaming with Converse API
def stream_converse(model_id, messages, inference_config=None):
    if inference_config is None:
        inference_config = {}

    print("Streaming response (chunks will appear as they are received):\n")
    print("-" * 80)

    full_response = ""

    try:
        response = bedrock.converse_stream(
            modelId=model_id,
            messages=messages,
            inferenceConfig=inference_config
        )
        response_stream = response.get('stream')
        if response_stream:
            for event in response_stream:

                if 'messageStart' in event:
                    print(f"\nRole: {event['messageStart']['role']}")

                if 'contentBlockDelta' in event:
                    print(event['contentBlockDelta']['delta']['text'], end="")

                if 'messageStop' in event:
                    print(f"\nStop reason: {event['messageStop']['stopReason']}")

                if 'metadata' in event:
                    metadata = event['metadata']
                    if 'usage' in metadata:
                        print("\nToken usage")
                        print(f"Input tokens: {metadata['usage']['inputTokens']}")
                        print(
                            f":Output tokens: {metadata['usage']['outputTokens']}")
                        print(f":Total tokens: {metadata['usage']['totalTokens']}")
                    if 'metrics' in event['metadata']:
                        print(
                            f"Latency: {metadata['metrics']['latencyMs']} milliseconds")


            print("\n" + "-" * 80)
        return full_response

    except Exception as e:
        print(f"Error in streaming: {str(e)}")
        return None

# Let's try streaming a longer summary
streaming_request = [
    {
        "role": "user",
        "content": [
            {
                "text": f"""Please provide a detailed summary of the following text, explaining its key points and implications:

                {text_to_summarize}

                Make your summary comprehensive but clear.
                """
            }
        ]
    }
]

In [34]:
# Only run this when you're ready to see streaming output
streamed_response = stream_converse(
    MODELS["Claude 3.7 Sonnet"],
    streaming_request,
    inference_config={"temperature": 0.4, "maxTokens": 1000}
)

Streaming response (chunks will appear as they are received):

--------------------------------------------------------------------------------

Role: assistant
# Summary: Amazon Bedrock - Democratizing Access to Foundation Models

## Key Points

1. **Amazon Bedrock Introduction**: AWS has launched Amazon Bedrock, a new service providing API access to Foundation Models (FMs) from multiple providers including AI21 Labs, Anthropic, Stability AI, and Amazon itself.

2. **Purpose and Value Proposition**: Bedrock aims to democratize access to generative AI by offering the easiest way for customers to build and scale applications using foundation models, regardless of technical expertise.

3. **Model Offerings**: The service provides access to various powerful foundation models for both text and images, including Amazon's new Titan large language models (LLMs).

4. **Service Characteristics**:
   - Serverless experience
   - Scalable, reliable, and secure AWS managed service
   - No infrastr

# 3. Code Generation with FMs

In [35]:
code_generation_prompt = """
Create a Python function called get_weather that accepts a location as parameter. \
The function should return a dictionary containing weather data (condition, temperature, and humidity) for predefined cities.\
Use a mock data structure instead of actual API calls. Include New York, San Francisco, Miami, and Seattle as default cities.\
The return statement should look like the following: return weather_data.get(location, {"condition": "Unknown", "temperature": 0, "humidity": 0}).
Only return the function and no preamble or examples.
"""

converse_request = {
    "messages": [
        {
            "role": "user",
            "content": [
                {
                    "text": f"{code_generation_prompt}"
                }
            ]
        }
    ],
    "inferenceConfig": {
        "temperature": 0.0,
        "topP": 0.9,
        "maxTokens": 500
    }
}

try:
    response = bedrock.converse(
        modelId=MODELS["Claude 3.7 Sonnet"],
        messages=converse_request["messages"],
        inferenceConfig=converse_request["inferenceConfig"]
    )

    # Extract the model's response
    claude_converse_response = response["output"]["message"]["content"][0]["text"]
    display_response(claude_converse_response, "Claude 3.7 Sonnet (Converse API)")
except botocore.exceptions.ClientError as error:
    if error.response['Error']['Code'] == 'AccessDeniedException':
        print(f"\x1b[41m{error.response['Error']['Code']}: {error.response['Error']['Message']}\x1b[0m")
        print("Please ensure you have the necessary permissions for Amazon Bedrock.")
    else:
        raise error

### Response from Claude 3.7 Sonnet (Converse API)

```python
def get_weather(location):
    weather_data = {
        "New York": {"condition": "Cloudy", "temperature": 72, "humidity": 65},
        "San Francisco": {"condition": "Foggy", "temperature": 62, "humidity": 80},
        "Miami": {"condition": "Sunny", "temperature": 85, "humidity": 75},
        "Seattle": {"condition": "Rainy", "temperature": 58, "humidity": 90}
    }
    return weather_data.get(location, {"condition": "Unknown", "temperature": 0, "humidity": 0})
```


--------------------------------------------------------------------------------



In [38]:
def get_weather(location):
    weather_data = {
        "New York": {"condition": "Cloudy", "temperature": 72, "humidity": 65},
        "San Francisco": {"condition": "Foggy", "temperature": 62, "humidity": 80},
        "Miami": {"condition": "Sunny", "temperature": 85, "humidity": 75},
        "Seattle": {"condition": "Rainy", "temperature": 58, "humidity": 90}
    }
    return weather_data.get(location, {"condition": "Unknown", "temperature": 0, "humidity": 0})

get_weather("New York")

{'condition': 'Cloudy', 'temperature': 72, 'humidity': 65}

# 4. Function Calling with Amazon Bedrock

Modern LLMs like Claude go beyond generating free-form text — they can also reason about when external tools or functions should be used to better answer user questions. This capability, known as function calling (or tool use), enables the model to decide which function to call, when to call it, and what parameters to provide — but importantly, the model does not execute the function itself.

Instead, the model returns a well-structured response (typically in JSON format) that describes the intended function call. It’s then up to your application to detect this output, execute the requested function (such as calling an API or querying a database), and pass the result back to the model — allowing it to generate a final, user-friendly response that incorporates real-world data.

Function calling is especially useful when building LLM-powered applications that need access to dynamic, external information — for example, retrieving real-time weather data, which is exactly what we’ll demonstrate in the this section.

### 4.1 Function Calling Flow
Amazon Bedrock natively supports function calling through its Converse API, which provides a consistent way to give LLMs access to tools or functions that live outside the model.

The typical function calling flow looks like this:

- **Step 1 — Function Recognition**. When the model (e.g., Claude 3.7) identifies that a tool should be used to answer a user query (e.g., "What's the weather in Seattle?"), it returns a structured response indicating:
  - The function name to call (e.g., get_weather)
  - The required input parameters (e.g., location="Seattle")

- **Step 2 — Function Execution** (by your application). The application is responsible for:
  - Executing the requested function
  - Capturing the output (e.g., current weather data)
  - Passing the result back to the model

- **Step 3 — Final Response Generation.** The LLM uses the function's output to generate a natural language response for the user.

### 4.2 Example: Implementation of Weather Function Calling with Bedrock's Converse API

we’ll integrate a simple get_weather function with Claude 3.7 using the Converse API. The tool specification we provide to Claude defines:

- The function name: get_weather
- The function's purpose: "Retrieve weather for a given location"
- The required input parameters: location (string)

Example Workflow:
- User: "What's the weather in Seattle?"
↓
- LLM: Returns function call request for `get_weather("Seattle")`
↓
- Application: Executes weather lookup, and feeds the result back to the LLM
↓
- LLM: Generates response using actual weather data

In [41]:
def get_weather(location):
    weather_data = {
        "New York": {"condition": "Cloudy", "temperature": 72, "humidity": 65},
        "San Francisco": {"condition": "Foggy", "temperature": 62, "humidity": 80},
        "Miami": {"condition": "Sunny", "temperature": 85, "humidity": 75},
        "Seattle": {"condition": "Rainy", "temperature": 58, "humidity": 90}
    }
    return weather_data.get(location, {"condition": "Unknown", "temperature": 0, "humidity": 0})


weather_tool = {
    "tools": [
        {
            "toolSpec": {
                "name": "get_weather",
                "description": "Get current weather for a specific location",
                "inputSchema": {
                    "json": {
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city name to get weather for"
                            }
                        },
                        "required": ["location"]
                    }
                }
            }
        }
    ],
    "toolChoice": {
        "auto": {}  # Let the model decide when to use the tool
    }
}

function_request = {
    "messages": [
        {
            "role": "user",
            "content": [
                {
                    "text": "What's the weather like in San Francisco right now? And what should I wear?"
                }
            ]
        }
    ],
    "inferenceConfig": {
        "temperature": 0.0,  # Use 0 temperature for deterministic function calling
        "maxTokens": 500
    }
}

With the tool specification and the function_request we can now invoke the model and take a look it it's response. Observe how it correctly fills out the "toolUse" parameter in its response.

In [42]:
response = bedrock.converse(
    modelId=MODELS["Claude 3.7 Sonnet"],
    messages=function_request["messages"],
    inferenceConfig=function_request["inferenceConfig"],
    toolConfig=weather_tool
)
print(json.dumps(response, indent=2))

{
  "ResponseMetadata": {
    "RequestId": "499a1f32-06dd-4b03-b001-1ca72c3392aa",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Tue, 02 Dec 2025 13:48:51 GMT",
      "content-type": "application/json",
      "content-length": "570",
      "connection": "keep-alive",
      "x-amzn-requestid": "499a1f32-06dd-4b03-b001-1ca72c3392aa"
    },
    "RetryAttempts": 0
  },
  "output": {
    "message": {
      "role": "assistant",
      "content": [
        {
          "text": "I can check the current weather in San Francisco for you and provide clothing recommendations based on that information."
        },
        {
          "toolUse": {
            "toolUseId": "tooluse_8f3RHsKNTuGNvHkKcp8_2g",
            "name": "get_weather",
            "input": {
              "location": "San Francisco"
            },
            "type": "tool_use"
          }
        }
      ]
    }
  },
  "stopReason": "tool_use",
  "usage": {
    "inputTokens": 404,
    "outputTokens": 75,
    "tot

In [43]:
def handle_function_calling(model_id, request, tool_config):
    try:
        # Step 1: Send initial request
        response = bedrock.converse(
            modelId=model_id,
            messages=request["messages"],
            inferenceConfig=request["inferenceConfig"],
            toolConfig=tool_config
        )

        # Check if the model wants to use a tool (check the correct response structure)
        content_blocks = response["output"]["message"]["content"]
        has_tool_use = any("toolUse" in block for block in content_blocks)

        if has_tool_use:
            # Find the toolUse block
            tool_use_block = next(block for block in content_blocks if "toolUse" in block)
            tool_use = tool_use_block["toolUse"]
            tool_name = tool_use["name"]
            tool_input = tool_use["input"]
            tool_use_id = tool_use["toolUseId"]

            # Step 2: Execute the tool
            if tool_name == "get_weather":
                tool_result = get_weather(tool_input["location"])
            else:
                tool_result = {"error": f"Unknown tool: {tool_name}"}

            # Step 3: Send the tool result back to the model
            updated_messages = request["messages"] + [
                {
                    "role": "assistant",
                    "content": [
                        {
                            "toolUse": {
                                "toolUseId": tool_use_id,
                                "name": tool_name,
                                "input": tool_input
                            }
                        }
                    ]
                },
                {
                    "role": "user",
                    "content": [
                        {
                            "toolResult": {
                                "toolUseId": tool_use_id,
                                "content": [
                                    {
                                        "json": tool_result
                                    }
                                ],
                                "status": "success"
                            }
                        }
                    ]
                }
            ]

            # Step 4: Get final response
            final_response = bedrock.converse(
                modelId=model_id,
                messages=updated_messages,
                inferenceConfig=request["inferenceConfig"],
                toolConfig=tool_config
            )

            # Extract text from the correct response structure
            final_text = ""
            for block in final_response["output"]["message"]["content"]:
                if "text" in block:
                    final_text = block["text"]
                    break

            return {
                "tool_call": {"name": tool_name, "input": tool_input},
                "tool_result": tool_result,
                "final_response": final_text
            }
        else:
            # Model didn't use a tool, just return the text response
            text_response = ""
            for block in content_blocks:
                if "text" in block:
                    text_response = block["text"]
                    break

            return {
                "final_response": text_response
            }

    except Exception as e:
        print(f"Error in function calling: {str(e)}")
        return {"error": str(e)}

In [44]:
function_result = handle_function_calling(
    MODELS["Claude 3.7 Sonnet"],
    function_request,
    weather_tool
)

# Display the results
if "error" not in function_result:
    if "tool_call" in function_result:
        print(f"Tool Call: {function_result['tool_call']['name']}({function_result['tool_call']['input']})")
        print(f"Tool Result: {function_result['tool_result']}")

    display_response(function_result["final_response"], "Claude 3.7 Sonnet (Function Calling)")
else:
    print(f"Error: {function_result['error']}")

Tool Call: get_weather({'location': 'San Francisco'})
Tool Result: {'condition': 'Foggy', 'temperature': 62, 'humidity': 80}


### Response from Claude 3.7 Sonnet (Function Calling)

Currently in San Francisco, it's 62°F with foggy conditions and 80% humidity.

For clothing recommendations based on this weather:
- A light jacket or sweater would be appropriate for the mild temperature
- Consider layering since San Francisco weather can change throughout the day
- The fog might make it feel a bit cooler than the temperature suggests
- Comfortable walking shoes are always good for San Francisco's hills
- You might want to bring a hat or light scarf if you're sensitive to the damp foggy air

San Francisco is known for its microclimates, so if you're planning to visit different neighborhoods, having an extra layer available is always a good idea.


--------------------------------------------------------------------------------

